In [80]:
import pandas as pd
import pandas_ta as ta
import os
from dotenv import load_dotenv
from datetime import timedelta

from core.services.timescale_client import TimescaleClient

load_dotenv()

ts_client = TimescaleClient(os.getenv("TIMESCALE_HOST", "localhost"))
await ts_client.connect()

In [81]:
available_pairs = await ts_client.get_available_pairs()
available_pairs

[('binance_perpetual', '1000000MOG-USDT'),
 ('binance_perpetual', '1000BONK-USDT'),
 ('binance_perpetual', '1000CAT-USDT'),
 ('binance_perpetual', '1000FLOKI-USDT'),
 ('binance_perpetual', '1000LUNC-USDT'),
 ('binance_perpetual', '1000PEPE-USDT'),
 ('binance_perpetual', '1000RATS-USDT'),
 ('binance_perpetual', '1000SATS-USDT'),
 ('binance_perpetual', '1000SHIB-USDT'),
 ('binance_perpetual', '1000XEC-USDT'),
 ('binance_perpetual', '1INCH-USDT'),
 ('binance_perpetual', '1MBABYDOGE-USDT'),
 ('binance_perpetual', 'AAVE-USDT'),
 ('binance_perpetual', 'ACE-USDT'),
 ('binance_perpetual', 'ACH-USDT'),
 ('binance_perpetual', 'ADA-USDT'),
 ('binance_perpetual', 'AERGO-USDT'),
 ('binance_perpetual', 'AEVO-USDT'),
 ('binance_perpetual', 'AGLD-USDT'),
 ('binance_perpetual', 'AI-USDT'),
 ('binance_perpetual', 'ALGO-USDT'),
 ('binance_perpetual', 'ALICE-USDT'),
 ('binance_perpetual', 'ALPACA-USDT'),
 ('binance_perpetual', 'ALPHA-USDT'),
 ('binance_perpetual', 'ALT-USDT'),
 ('binance_perpetual', 'AMB-

In [82]:
intervals = ["1m", "3m", "5m", "15m", "1h"]

status = []
for connector_name, trading_pair in available_pairs:
    for interval in intervals:
        try:
            candles = await ts_client.get_candles(connector_name, trading_pair, interval=interval)
            candles_status = {
                "connector_name": connector_name,
                "trading_pair": trading_pair,
                "start": candles.data.timestamp.min(),
                "end": candles.data.timestamp.max()
            }
            status.append(candles_status)
        except Exception as e:
            print(f"Error while parsing {trading_pair} at {interval}: {e}")
            continue

Error while parsing KLAY-USDT at 1m: relation "binance_perpetual_klay_usdt_1m" does not exist
Error while parsing KLAY-USDT at 3m: relation "binance_perpetual_klay_usdt_3m" does not exist
Error while parsing KLAY-USDT at 5m: relation "binance_perpetual_klay_usdt_5m" does not exist
Error while parsing KLAY-USDT at 15m: relation "binance_perpetual_klay_usdt_15m" does not exist
Error while parsing KLAY-USDT at 1h: relation "binance_perpetual_klay_usdt_1h" does not exist
Error while parsing UNFI-USDT at 1m: relation "binance_perpetual_unfi_usdt_1m" does not exist
Error while parsing UNFI-USDT at 3m: relation "binance_perpetual_unfi_usdt_3m" does not exist
Error while parsing UNFI-USDT at 5m: relation "binance_perpetual_unfi_usdt_5m" does not exist
Error while parsing UNFI-USDT at 15m: relation "binance_perpetual_unfi_usdt_15m" does not exist
Error while parsing UNFI-USDT at 1h: relation "binance_perpetual_unfi_usdt_1h" does not exist


In [85]:
df = pd.DataFrame(status)
df["start"] = pd.to_datetime(df["start"], unit="s")
df["end"] = pd.to_datetime(df["end"], unit="s")
df.sort_values("start")

,connector_name,trading_pair,start,end
294,binance_perpetual,BOME-USDT,2024-10-10 00:00:00,2024-11-09 16:00:00
850,binance_perpetual,METIS-USDT,2024-10-10 00:00:00,2024-11-09 16:21:00
851,binance_perpetual,METIS-USDT,2024-10-10 00:00:00,2024-11-09 16:21:00
1380,binance_perpetual,VIDT-USDT,2024-10-10 00:00:00,2024-11-09 16:21:00
852,binance_perpetual,METIS-USDT,2024-10-10 00:00:00,2024-11-09 16:20:00
...,...,...,...,...
1498,binance_perpetual,ZRX-USDT,2024-11-08 17:00:00,2024-11-09 16:15:00
744,binance_perpetual,KNC-USDT,2024-11-08 17:00:00,2024-11-09 16:00:00
743,binance_perpetual,KNC-USDT,2024-11-08 17:00:00,2024-11-09 16:15:00
733,binance_perpetual,KDA-USDT,2024-11-08 17:00:00,2024-11-09 16:15:00


In [ ]:

def calculate_global_screener_metrics(candles_df: pd.DataFrame, connector_name: str, trading_pair: str):
    df = candles_df.copy()
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
    df = df.set_index("timestamp")
    global_metrics = {}
    
    # 1. Trading Pair (assuming static)
    global_metrics['connector_name'] = connector_name
    global_metrics['trading_pair'] = trading_pair
    
    # 2. Price Analysis
    # Describe price statistics
    global_metrics['price'] = df['close'].describe().to_dict()
    
    # Helper function for percent change, focusing only on the relevant column
    def percent_change(series, period):
        shifted = series.shift(period)
        if shifted.iloc[-1] == 0 or pd.isna(shifted.iloc[-1]):
            return None
        return (series.iloc[-1] - shifted.iloc[-1]) / shifted.iloc[-1]

    # Price Change Over Periods (% CBO 24h, 1w, 1m)
    price_resampled = df['close'].resample('1d').last()  # Daily resampling for consistent periods
    global_metrics['price_cbo'] = {
        '24h': percent_change(price_resampled, 1),
        '1w': percent_change(price_resampled, 7),
        '1m': percent_change(price_resampled, 28)
    }
    
    # 3. Volume Analysis
    # 24h Volume USD
    last_24h = df.loc[df.index >= (df.index[-1] - timedelta(hours=24)), 'volume'].sum()
    global_metrics['volume_24h'] = last_24h

    # Volume % CBO for Different Periods
    volume_resampled = df['volume'].resample('1d').sum()
    global_metrics['volume_cbo'] = {
        '24h': percent_change(volume_resampled, 1),
        '1w': percent_change(volume_resampled, 7),
        '1m': percent_change(volume_resampled, 30)
    }
    
    return global_metrics

In [72]:
def calculate_interval_screener_metrics(candles_df: pd.DataFrame, connector_name: str, trading_pair: str, interval: str):
    screener_metrics = {}
    
    df = candles_df.copy()
    df['atr_24h'] = ta.atr(df['high'], df['low'], df['close'], length=24)
    df['atr_1w'] = ta.atr(df['high'], df['low'], df['close'], length=7 * 24)
    df['natr_24h'] = ta.natr(df['high'], df['low'], df['close'], length=24)
    df['natr_1w'] = ta.natr(df['high'], df['low'], df['close'], length=7 * 24)
    screener_metrics['natr'] = df[['natr_24h', 'natr_1w']].describe().to_dict()
    
    # Bollinger Bands Width (50, 100, 200 / 2.0)
    for window in [50, 100, 200]:
        bb = ta.bbands(df['close'], length=window, std=2)
        screener_metrics[f'bb_width_{window}'] = bb[f'BBB_{window}_2.0'].mean() / 200
    return screener_metrics

{'1h': {'natr': {'natr_24h': {'count': 714.0,
    'mean': 1.6350947149492814,
    'std': 0.35077719527662155,
    'min': 0.9909145510227584,
    '25%': 1.3576595214276117,
    '50%': 1.6208881162867221,
    '75%': 1.8590090659452223,
    'max': 2.9076928502906427},
   'natr_1w': {'count': 570.0,
    'mean': 1.618303260062746,
    'std': 0.09807154183624879,
    'min': 1.394299140784235,
    '25%': 1.546543352319969,
    '50%': 1.6322631599670339,
    '75%': 1.6879250440888005,
    'max': 1.836680800499064}},
  'bb_width_50': 0.05372805124209204,
  'bb_width_100': 0.0826735803418001,
  'bb_width_200': 0.10278118257370272}}

In [77]:
intervals = ["1m", "3m", "5m", "15m", "1h"]

connector_name = "binance_perpetual"
trading_pair = "1000FLOKI-USDT"
candles = await ts_client.get_candles(connector_name, trading_pair, interval="1h")

global_metrics = calculate_global_screener_metrics(candles_df=candles.data,
                                                   connector_name=connector_name, 
                                                   trading_pair=trading_pair)

interval_screener_metrics = {}
for selected_interval in intervals:
    candles = await ts_client.get_candles(connector_name, trading_pair, interval=selected_interval)
    interval_screener_metrics[selected_interval] = calculate_interval_screener_metrics(candles_df=candles.data, 
                                                                                       connector_name=connector_name, 
                                                                                       trading_pair=trading_pair, 
                                                                                       interval=selected_interval)
screener_metrics = global_metrics.update(interval_screener_metrics)

In [78]:
screener_metrics    